## Import modules

In [1]:
import pickle
import json

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorchtools
import glob as gl
import random
import os
import time
import itertools

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.datasets import make_regression
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import log_loss
from numpy import hstack
from numpy import vstack

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import warnings
warnings.filterwarnings(action='ignore')

## Data loading

In [2]:
def find_directory(foldername, filename = None, back_num = 0):
    cur = os.getcwd()
    for i in range(back_num):
        cur = os.path.abspath(os.path.join(cur, os.pardir))
    for folder in foldername:
        cur = os.path.join(cur, folder)
    if not os.path.exists(cur):
        os.makedirs(cur)
        print(f'{cur} created')
    if filename != None:
        cur = os.path.join(cur, filename)
    return cur

os.getcwd()
find_directory(back_num = 1, foldername = ['Dataset'], filename = 'bat_dict.pkl')

'C:\\Users\\User\\Desktop\\LIB\\CNN work\\Variable time length\\All dataset'

'C:\\Users\\User\\Desktop\\LIB\\CNN work\\Variable time length\\Dataset\\bat_dict.pkl'

In [3]:
bat_dict_add_1_2 = find_directory(back_num = 2, foldername = ['Dataset'], filename = 'bat_1_2_dict_VITQcQd.pkl')
with open(bat_dict_add_1_2, 'rb') as tf:
    bat_dict_1_2 = pickle.load(tf)

bat_sel_dict = bat_dict_1_2
max_time = 120

In [4]:
# Tensor dimension: (cell, cycle, time, variable)
num_cells = len(bat_sel_dict.keys())
regression_cycles = [45, 65, 85, 100]

# variable = [Qc, I, V, T, Qd]
num_variables = len(bat_sel_dict['b1c0']['1']['1.0'].keys())

for reg in regression_cycles:
    #'''RNN 가변길이 다룰수 있게?? 고민필요'''
    globals()[f'x_tensor_{reg}'] = torch.zeros(num_cells, reg, max_time, num_variables)
    cell_index = 1
    for cell in bat_sel_dict.keys():
        for cycle in range(1, reg+1):
            for time in bat_sel_dict[cell][str(cycle)].keys():
    #             print(bat_sel_dict[cell][str(cycle)][str(time)])
                value_list = [i for i in bat_sel_dict[cell][str(cycle)][str(time)].values()]
    #             print(value_list)
                globals()[f'x_tensor_{reg}'][cell_index-1, cycle-1, int(float(time)*max_time/60), :] = torch.FloatTensor(value_list)

        cell_index += 1

    globals()[f'x_tensor_{reg}'].size()

num_variables = x_tensor_100.shape[3]
time_length = x_tensor_100.shape[2]
# (cycle_index-1, cycle-1, time-1, variable(Qc, I, V, T, Qd))
# x_tensor[123, 99, 30, :]

torch.Size([124, 45, 120, 5])

torch.Size([124, 65, 120, 5])

torch.Size([124, 85, 120, 5])

torch.Size([124, 100, 120, 5])

In [5]:
y_df_add = find_directory(back_num = 2, foldername = ['Dataset', 'y_df.csv'])
y_df = pd.read_csv(y_df_add)

# 2. Regression problem
# KO, KP, CL
y_cl_numpy = y_df['Cycle life'].to_numpy()
y_kp_numpy = y_df['kneepoints'].to_numpy()
y_ko_numpy = y_df['kneeonsets'].to_numpy()
# y_ko_numpy.shape

y_regression_numpy = y_ko_numpy

y_ko_tensor = torch.FloatTensor(y_ko_numpy)
y_ko_tensor.size()

torch.Size([124])

In [6]:
def setRandomSeed(random_seed=0):
    os.environ['PYTHONHASHSEED'] = str(random_seed)
    torch.manual_seed(random_seed) # torch 
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
    torch.backends.cudnn.deterministic = True # cudnn
    torch.backends.cudnn.benchmark = False # cudnn
    np.random.seed(random_seed) # numpy
    random.seed(random_seed) # random

In [7]:
x_tensor_100.shape

torch.Size([124, 100, 120, 5])

### Dataloading as 2D CNN's method

In [8]:
# y: no scaling
class dataPrep_RNN_CNN(Dataset):  
    def __init__(self, x_tensor, y_tensor, batch_size, scaler, purpose : str, test_split: float):
        setRandomSeed(random_seed = 100)
        self.xdata = torch.permute(x_tensor, (0, 3, 2, 1))
        self.ydata = y_tensor
        
        self.batch_size = batch_size
                
        self.xscaler = scaler()
        self.purpose = purpose
        
        self.test_split = test_split
        
        self.train_size, self.val_size, self.test_size = self.get_splits()
        
        # Tensor dimension: (cell, variable, time, cycle)
        # If problem emerges, check this part first.
        for i in range(len(self.xdata)):
            for j in range(self.xdata.shape[1]):
                temp = np.expand_dims(self.xscaler.fit_transform(self.xdata[i,j,:,:]), axis=0)
                if j ==0:
                    temp2 = temp
                else:
                    temp2 = np.vstack((temp2,temp))
            temp2 = np.expand_dims(temp2, axis=0)
            if i==0:
                self.xdata_scaled = temp2
            else:
                self.xdata_scaled = np.vstack((self.xdata_scaled, temp2))

        self.ydata_scaled = self.ydata
        
        self.x_train_scaled, self.x_test_scaled, self.y_train_scaled, self.y_test_scaled = train_test_split(self.xdata_scaled, 
                                                                                                            self.ydata_scaled, 
                                                                                                            test_size = self.test_size, 
                                                                                                            shuffle = True, 
                                                                                                            random_state=100)
        
        self.x_train_scaled, self.x_val_scaled, self.y_train_scaled, self.y_val_scaled = train_test_split(self.x_train_scaled,
                                                                                                         self.y_train_scaled,
                                                                                                         test_size = self.val_size, 
                                                                                                         shuffle = True, 
                                                                                                         random_state = 100)
        
        print(self.x_train_scaled.shape, self.x_val_scaled.shape, self.x_test_scaled.shape,
              self.y_train_scaled.shape, self.y_val_scaled.shape, self.y_test_scaled.shape)
        
        self.train = list(zip(self.x_train_scaled, self.y_train_scaled))
        self.val = list(zip(self.x_val_scaled, self.y_val_scaled))
        self.test = list(zip(self.x_test_scaled, self.y_test_scaled))
                
        self.train_dataloader = DataLoader(self.train, batch_size = self.batch_size, shuffle=True)
        self.val_dataloader = DataLoader(self.val, batch_size = self.batch_size, shuffle=True)
        self.test_dataloader = DataLoader(self.test, batch_size = self.batch_size, shuffle=True)
            
    # split the datasets into training and testing
    def get_splits(self):
        # test size
        test_size = int(self.test_split*len(self.xdata))
        # val size
        val_size = int(0.2*(len(self.xdata) - test_size))
        # train_size
        train_size = len(self.xdata) - val_size - test_size
        return train_size, val_size, test_size
    
    # Returns dataloaders for training and validation datasets
    def train_data(self):
        return self.train_dataloader, self.val_dataloader
    
    # Returns dataloader for test datasets
    def test_data(self):
        return self.test_dataloader
    
    def scaler(self, purpose):
#         if purpose != 'classification':
#             return self.xscaler, self.yscaler
#         else:
        return self.xscaler
    
    # finding length of x 
    def __len__(self):
        return len(self.X)
    
    # indexing rows for calling
    def __getitem__(self, idx):
        return [self.X[idx], self.y[idx]]


In [9]:
# reg_dataset = dataPrep1D(x_tensor, y_ko_tensor, batch_size = 16, scaler = MinMaxScaler, purpose = 'regression', test_split = 0.2)
# reg_train_dataloader, reg_val_dataloader = reg_dataset.train_data()
# reg_test_dataloader = reg_dataset.test_data()
# reg_xscaler = reg_dataset.scaler('regression')

regression_cycles = [45, 65, 85, 100]
# regression_cycles = [100]


# x와 y가 1대1 대응이어야 dataloader를 만들 수 있어서 2D array 그대로 놓고 학습시 tensor transform 예정
for reg in regression_cycles:
    globals()[f'reg_dataset_{reg}'] = dataPrep_RNN_CNN(globals()[f'x_tensor_{reg}'], y_ko_tensor, batch_size = 4, scaler = MinMaxScaler, purpose =  'regression', test_split = 0.2)
    globals()[f'reg_train_dataloader_{reg}'], globals()[f'reg_val_dataloader_{reg}'] = globals()[f'reg_dataset_{reg}'].train_data()
    globals()[f'reg_test_dataloader_{reg}'] = globals()[f'reg_dataset_{reg}'].test_data()
    globals()[f'reg_xscaler_{reg}'] = globals()[f'reg_dataset_{reg}'].scaler('regression')

(80, 5, 120, 45) (20, 5, 120, 45) (24, 5, 120, 45) torch.Size([80]) torch.Size([20]) torch.Size([24])
(80, 5, 120, 65) (20, 5, 120, 65) (24, 5, 120, 65) torch.Size([80]) torch.Size([20]) torch.Size([24])
(80, 5, 120, 85) (20, 5, 120, 85) (24, 5, 120, 85) torch.Size([80]) torch.Size([20]) torch.Size([24])
(80, 5, 120, 100) (20, 5, 120, 100) (24, 5, 120, 100) torch.Size([80]) torch.Size([20]) torch.Size([24])


In [10]:
# train_features, train_labels = next(iter(cla_train_dataloader))
# print(f"Classification case_Inputs batch shape(Batch size, num_vars, timestep, cycles): {train_features.size()}")
# print(f"Classification case_Outputs batch shape(Batch size): {train_labels.size()}")

reg = 100
train_features, train_labels = next(iter(globals()[f'reg_train_dataloader_{reg}']))
print(f'reg = {reg}')
print(f"Regression case_Inputs batch shape(Batch size, num_vars, timestep, cycles): {train_features.size()}")
print(f"Regression case_Outputs batch shape(Batch size): {train_labels.size()}")

train_features.float().type()

reg = 100
Regression case_Inputs batch shape(Batch size, num_vars, timestep, cycles): torch.Size([4, 5, 120, 100])
Regression case_Outputs batch shape(Batch size): torch.Size([4])


'torch.FloatTensor'

## Define Model Class 

In [11]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [12]:
class RNNcell(nn.Module):
    def __init__(self, model, input_size, hidden_size, dropout = 0.1, bidirectional = False, num_layer=1):
        super(RNNcell, self).__init__()
        
        if torch.cuda.is_available():
            self.device = 'cuda'
        else:
            self.device = 'cpu'
            
        self.model = model
        self.hidden_size = hidden_size
        self.num_layer = num_layer
        self.bidirectional = bidirectional
        
        self.D = 1 + self.bidirectional
        self.batch_size = 4
        
        #Batch size * cycles, timestep, input_size
        if self.model.endswith('RNN'):
            self.rnn = nn.RNN(input_size, hidden_size, num_layer, batch_first = True, dropout = dropout, bidirectional = bidirectional).to(self.device)
        elif self.model.endswith('LSTM'):
            self.rnn = nn.LSTM(input_size, hidden_size, num_layer, batch_first = True, dropout = dropout, bidirectional = bidirectional).to(self.device)
        elif self.model.endswith('GRU'):
            self.rnn = nn.GRU(input_size, hidden_size, num_layer, batch_first = True, dropout = dropout, bidirectional = bidirectional).to(self.device)
            
        for name, param in self.rnn.named_parameters():
            if name.startswith('weight'):
                nn.init.xavier_uniform(param)
            else:
                nn.init.normal(param)
        
    def forward(self, x):
        # x made from (cell * n_cy, timestep(seq_len), num_vars)
        self.batch_size = x.size(0)
        
        D = 1 + self.bidirectional
        #out: Containing output features h_t from the last layer of LSTM for each t(if bidirectional ==True: concat of forward and backward hidden states)
        #h_n: Containing final hidden state in the sequence
        h0 = torch.zeros(D*self.num_layer, self.batch_size, self.hidden_size).to(self.device)
        if self.model.endswith('LSTM'):
            c0 = torch.zeros(D*self.num_layer, self.batch_size, self.hidden_size).to(self.device)
            # c_n: Containing final cell state in the sequence
            out, (hn, cn) = self.rnn(x, (h0, c0))
        else:
            out, hn = self.rnn(x, h0.detach())
        return out, hn

In [23]:
class SelfAttention(nn.Module):
    def __init__(self, hidden_size, num_heads):
        super(SelfAttention, self).__init__()
        self.hidden_size = hidden_size
        self.num_heads = num_heads
        self.head_size = hidden_size // num_heads
        
        self.query = nn.Linear(hidden_size, hidden_size)
        self.key = nn.Linear(hidden_size, hidden_size)
        self.value = nn.Linear(hidden_size, hidden_size)
        self.softmax = nn.Softmax(dim=-1)
        
    def forward(self, inputs):
        # inputs shape: [batch_size, seq_len, hidden_size]
        batch_size, seq_len, hidden_size = inputs.size()
        
        # Compute query, key, and value matrices
        query = self.query(inputs).view(batch_size, seq_len, self.num_heads, self.head_size).transpose(1, 2)
        key = self.key(inputs).view(batch_size, seq_len, self.num_heads, self.head_size).transpose(1, 2)
        value = self.value(inputs).view(batch_size, seq_len, self.num_heads, self.head_size).transpose(1, 2)
        
        # Compute scaled dot-product attention scores
        # attention_scores: [batch_size, num_heads, seq_len, seq_len]
        attention_scores = torch.matmul(query, key.transpose(-2, -1)) / torch.sqrt(torch.tensor(self.head_size, dtype=torch.float32))
        attention_probs = self.softmax(attention_scores)
        
        # Apply attention to the value matrix and concatenate heads
        # context: [batch_size, num_heads, seq_len, head_size]=>[batch_size, seq_len, num_heads, head_size] 
        # => [batch_size, seq_len, num_heads * head_size]
        context = torch.matmul(attention_probs, value).transpose(1, 2).contiguous().view(batch_size, seq_len, -1)
#         context = nn.Linear(hidden_size, 1)(context)
        return attention_probs, context

In [14]:
class RNN_TA_CA_1DCNN(nn.Module):
    def __init__(self, input_size, num_time, num_cycles, rnn1, bi1, hid1, nhead2, fil2, pool2, npool2, fsize2, psize2, mids, dr1 = 0.1, dr2 = 0.1, di2 = 1, st2 = 1, pad2 = 0):
        super(RNN_TA_CA_1DCNN, self).__init__()
        setRandomSeed()
        
        self.D1 = 1 + bi1
        
        self.rnn1 = RNNcell(rnn1, input_size, hid1, dr1, bi1)
        
        if torch.cuda.is_available():
            self.device = 'cuda'
        else:
            self.device = 'cpu'
        
        # Overall parameters
        self.num_time = num_time
        self.num_cycles = num_cycles
        self.mids = mids
        
        # Pooling/Nonpooling parameters
        # Pooling/Nonpooling layers
        self.pool2 = pool2
        self.npool2 = npool2
        # Pooling size(Pooling layer)
        self.psize2 = psize2
        
        # Params for self-attention(ca)
        self.nh2 = nhead2
        
        # CNN hyperparameters
        self.fil2 = fil2
        self.fsize2 = fsize2
        self.di2 = di2
        self.st2 = st2
        # Padding for CNN
        self.pad2 = pad2
                
        # Critical values for the 1st CNN layer
        # Batch for different cells
        
        # Conv1d takes input dimension: [Batch_size, c_in(no. of in_channels )= hidden_size, l_in = num_cycles]
        self.in2 = self.D1*hid1
        self.out2 = self.fil2
        self.Lout = self.num_cycles
        
        # Conv_block1~P
        # Pooling layers
        for i in range(1, self.pool2+1):
#             print(f"i: {i}, out_channels: {self.out_channels}")
            globals()[f'self.conv_block{i}'] = nn.Sequential(
            nn.Conv1d(self.in2, self.out2, self.fsize2, self.st2, self.pad2),
            nn.BatchNorm1d(self.out2),
            nn.ReLU(),
            nn.MaxPool1d(self.psize2)
            ).to(self.device)

            self.in2 = self.out2
            self.out2 = self.out2*2
            self.Lout = int(int((self.Lout+2*self.pad2-self.di2*(self.fsize2-1)-1)/self.st2+1)/self.psize2)

        for j in range(self.pool2+1, self.pool2+self.npool2+1):
            globals()[f'self.conv_block{j}'] = nn.Sequential(
            nn.Conv1d(self.in2, self.out2, self.fsize2, self.st2, self.pad2),
            nn.BatchNorm1d(self.out2),
            nn.ReLU(),
            ).to(self.device)
            
            self.in2 = self.out2
            self.out2 = self.out2*2
            self.Lout = int((self.Lout+2*self.pad2-self.di2*(self.fsize2-1)-1)/self.st2+1)
        
        self.lin1 = nn.Linear(self.D1*hid1, 1)
        self.sm1 = nn.Softmax(dim = 2)
        self.relu = nn.ReLU()
        
        self.ca = SelfAttention(self.D1*hid1, self.nh2)
        
        self.fc = nn.Sequential(nn.Linear(self.in2*self.Lout, self.mids[0], bias=True), nn.ReLU(),
                       nn.Linear(self.mids[0], self.mids[1], bias = True), nn.ReLU(),
                       nn.Linear(self.mids[1], self.mids[2], bias = True))
        
        self.fc.apply(self.init_weights)
        
    def init_weights(self, m):
        self.m = m
        if type(self.m) == nn.Linear:
            nn.init.xavier_uniform_(self.m.weight)
        
    def forward(self, x):
        # Original: Batch size, num_vars, timestep, cycles=>Batch size * cycles, timestep, input_size
        x = torch.reshape(x, (x.size(0)*x.size(3), x.size(2), x.size(1))).to(device)
#         print("Batch size * cycles, timestep, input_size: ", x.size())
        out1, hn1 = self.rnn1(x)
        
        # hn의 batch size가 n_cy*k로 설정
        cell_batch_size = int(hn1.size(1)/self.num_cycles)

        # Reshape for input
        # out1: (batch_size = num_cell*num_cycle, timestep, D1*hid1)
        # outs : (batch_size(num_cell), num_cycle, timestep, D1*hid1)
        outs1 = torch.reshape(out1, (cell_batch_size, -1, out1.size(1), out1.size(2)))
        # ta: (batch_size, num_cycle, timestep, 1(D1*hid1->1))
        ta = self.sm1(self.lin1(outs1))
        # ta_outs: (batch_size, num_cycle, timestep, D1*hid1)=>(batch_size, D1*hid1, num_cycle, timestep)
        ta_outs = torch.permute(ta*outs1, (0, 3, 1, 2))
        # ct_vec: (batch_size, D1*hid1, num_cycle)
        ct_vec = torch.sum(ta_outs, -1)
        # ct_vec shape: [batch_size, num_cycle, D1*hid1(hidden_size)]
        # hidden_size = num_heads * head_size
        # ct_vec transform back: [batch_size, num_heads, num_cycle, head_size]=>[batch_size, num_cycle, num_heads, head_size] 
        # => [batch_size, num_cycle, num_heads * head_size(hidden_size)]
        # ca: matmul(query[batch_size, num_heads, num_cycle, head_size], key^T[batch_size, num_heads, head_size, num_cycle])
        # ca shape: [batch_size, num_heads, num_cycle, num_cycle]
        ca, ct_vec = self.ca(ct_vec.transpose(2, 1))
#         print(f"After self attention: ca.size: {ca.size()}, ct_vec.size: {ct_vec.size()}")
        # ct_vec shape: [batch_size, D1*hid1, num_cycle]
        ct_vec = ct_vec.transpose(2, 1)
        
        
        # input for 1d cnn: (batch_size, D1*hid1, num_cycle)
        for i in range(1, self.pool2+self.npool2+1):
            ct_vec = globals()[f'self.conv_block{i}'](ct_vec)
        # ct_vec from final convolutional layer: (batch_size, self.in2*self.Lout)
        ct_vec = ct_vec.view(ct_vec.size(0), -1)
        # final_out: (batch_size, 1)
        final_out = self.fc(ct_vec)
        
        return final_out.squeeze(), ta.squeeze(), ca

In [15]:
# Function for saving and loading of training history
def save_data(D3_array, filename):
    with open(filename,"wb") as dat_:
        pickle.dump(D3_array,dat_)
        
def load_data(filename):
    with open(filename,"rb") as ld:
        x_temp = pickle.load(ld)
    return x_temp

# Early stopping class
class EarlyStopping:
    """주어진 patience 이후로 validation loss가 개선되지 않으면 학습을 조기 중지"""
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt'):
        """
        Args:
            patience (int): validation loss가 개선된 후 기다리는 기간
                            Default: 7
            verbose (bool): True일 경우 각 validation loss의 개선 사항 메세지 출력
                            Default: False
            delta (float): 개선되었다고 인정되는 monitered quantity의 최소 변화
                            Default: 0
            path (str): checkpoint저장 경로
                            Default: 'checkpoint.pt'
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.best_epoch = 1
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path

    def __call__(self, epoch, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.best_model = self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
#             print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.best_epoch = epoch
            self.best_model = self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''validation loss가 감소하면 모델을 저장한다.'''
        if self.verbose:
            print(f'At epoch {self.best_epoch}: Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss
        return model

In [16]:
class train:
    def __init__(self, model: nn.Module, train_dataloader, val_dataloader, test_dataloader, epoch: int, learning_rate=0.01, patience = 5, verbose = False):
        super().__init__()
        #   Reprocudtion
        setRandomSeed(100)
        random_seed = 100
        if torch.cuda.is_available():
            torch.cuda.manual_seed_all(random_seed)
            torch.backends.cudnn.deterministic = True
            torch.backends.cudnn.benchmark = False
        os.environ['PYTHONHASHSEED'] = str(random_seed)
        torch.manual_seed(random_seed)
        np.random.seed(random_seed)

        if torch.cuda.is_available():
            self.device = 'cuda'
        else:
            self.device = 'cpu'
        
        self.model = model.to(self.device)
        self.train_dataloader = train_dataloader
        self.val_dataloader = val_dataloader
        self.test_dataloader = test_dataloader
        self.epoch = epoch
        self.learning_rate = learning_rate
        
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr = self.learning_rate)
        self.early_stopping = EarlyStopping(patience = patience, verbose = verbose)

        self.criterion = nn.MSELoss()
        self.name = ['train_rmse', 'val_rmse', 'test_rmse']
        
    def EvalModel(self):
        EvalPredictions, EvalActuals = list(), list()
        TestPredictions, TestActuals = list(), list()
        self.model.eval()
        with torch.no_grad():
            for n, (inputs, targets) in enumerate(self.val_dataloader):
                inputs = inputs.float().to(self.device)
                targets = targets.float().to(self.device)

                yhat, ta, ca = self.model(inputs)

                if self.device !='cpu':
                    yhat = yhat.to('cpu')
                    targets = targets.to('cpu')    

                yhat = yhat.detach().numpy()
                actual = targets.numpy()
                EvalPredictions.append(yhat)
                EvalActuals.append(actual)

            EvalPredictions, EvalActuals = vstack(EvalPredictions), vstack(EvalActuals)
            val_rmse = mean_squared_error(EvalActuals, EvalPredictions, squared = False)
            
            for n, (inputs, targets) in enumerate(self.test_dataloader):
                inputs = inputs.float().to(self.device)
                targets = targets.float().to(self.device)

                yhat, ta, ca = self.model(inputs)

                if self.device !='cpu':
                    yhat = yhat.to('cpu')
                    targets = targets.to('cpu')    

                yhat = yhat.detach().numpy()
                actual = targets.numpy()
                TestPredictions.append(yhat)
                TestActuals.append(actual)

            TestPredictions, TestActuals = vstack(TestPredictions), vstack(TestActuals)
            test_rmse = mean_squared_error(TestActuals, TestPredictions, squared = False)
            
            return val_rmse, test_rmse

    def TrainModel(self):
        loss_history = []
        self.model.train()
        for i in range(self.epoch):
            predictions, actuals = list(), list()
            for n, (inputs, targets) in enumerate(self.train_dataloader):
                self.model.train()
                inputs = inputs.float().to(self.device)
                targets = targets.float().to(self.device)

                self.optimizer.zero_grad()

                yhat, ta, ca = self.model(inputs)
                    
                loss = self.criterion(yhat, targets)
                loss.backward()
                self.optimizer.step()
                
                if self.device !='cpu':
                    yhat = yhat.to('cpu')
                    targets = targets.to('cpu')
                    
                yhat = yhat.detach().numpy()
                predictions.append(yhat)
                
                targets = targets.numpy()
                actuals.append(targets)

            # evaluation of entire train data
            predictions, actuals = vstack(predictions), vstack(actuals)
            
            train_rmse = mean_squared_error(actuals, predictions, squared = False)

            # evaluation on validation data
            val_rmse, test_rmse = self.EvalModel()
            early_factor = val_rmse
            
            # training loss history
            loss_history.append([train_rmse, val_rmse, test_rmse])

            self.early_stopping(i+1, early_factor, self.model)
            
            if self.early_stopping.early_stop:
                print("Early stopping at best epoch: ", self.early_stopping.best_epoch)
                self.best_loss = loss_history[self.early_stopping.best_epoch-1]
                loss_history = loss_history[:self.early_stopping.best_epoch]
                
                return self.model, self.best_loss, pd.DataFrame(loss_history, columns = self.name), ta, ca
            
        print(f"Ends at final epoch {self.epoch}")
        print(f"Best epoch: {self.early_stopping.best_epoch}")
        self.best_loss = loss_history[self.early_stopping.best_epoch-1]
        loss_history = loss_history[:self.early_stopping.best_epoch]
        
        return self.model, self.best_loss, pd.DataFrame(loss_history, columns = self.name), ta, ca
        
    def predict(self, data: torch.Tensor):
        return self.model(data)
    
    def train_val_test_plot_rmse(self, num_cycles, modelname, n_ep, patience, rnn1, hid1, fil2, pool2, npool2, fsize2, psize2, lr):
        dataloaders = [self.train_dataloader, self.val_dataloader, self.test_dataloader]

        with torch.no_grad():
            model.eval()

            model_rmse = list()
            dset_index = 0
            for dataloader in dataloaders:
                if dset_index ==0:
                    dset = 'train'
                elif dset_index == 1:
                    dset = 'val'
                else:
                    dset = 'test'

                figadd = find_directory(back_num = 0, foldername = [f'{num_cycles} cycles', f'Depth Test_col_{n_ep}_{patience}', modelname, f'{rnn1}_1D CNN', 'train_history'], 
                                             filename = f'{rnn1}_hidden_{hid1}_n_fil_{fil2}_pool_{pool2}_npool_{npool2}_fsize_{fsize2}_psize_{psize2}_lr_1_{int(1/lr)}.jpg')

                predictions, actuals = list(), list()
                for i, (test_input, test_target) in enumerate(dataloader):
                    test_input = test_input.float().to(self.device)
                    test_target = test_target.float().to(self.device)
                    test_yhat, test_ta = model(test_input)

                    if self.device != 'cpu':
                        test_yhat = test_yhat.to('cpu')
                        test_target = test_target.to('cpu')

                    test_yhat = test_yhat.detach().tolist()
                    predictions += test_yhat
                    test_target = test_target.tolist()
                    actuals += test_target

                pred_acc, targ_acc = np.array(predictions), np.array(actuals)

                rmse = mean_squared_error(actuals, predictions, squared = False)
                model_rmse.append(rmse)

                plt.figure(figsize=[10,4])
                plt.rcParams["font.size"] = "12"
                plt.plot(pred_acc, label='Prediction')
                plt.plot(range(len(targ_acc)), targ_acc, label = 'Actual')
                plt.xticks(range(0, len(targ_acc) + 1, int(len(targ_acc)/5)))
                plt.title(f"{rnn1}_hid_{hid1}_nfil_{fil2}_pool_{pool2}_npool_{npool2}_fsize_{fsize2}_psize_{psize2}: RMSE={rmse}")
                plt.xlabel('cell')
                plt.ylabel('Lifespan(cycle)')
                plt.legend()
                plt.savefig(figadd)
                plt.close()

                dset_index += 1

        return model_rmse

## Implementation

In [17]:
def history_state_dict_add(num_cycles, modelname, n_ep, patience, rnn1, hid1, nh2, fil2, pool2, npool2, fsize2, psize2, lr):
    history_add = find_directory(back_num = 0, foldername = [f'{num_cycles} cycles', f'Depth Test_col_{n_ep}_{patience}', modelname, f'{rnn1}_1D CNN', 'train_history'], 
                                             filename = f'{rnn1}_hidden_{hid1}_nh_{nh2}_n_fil_{fil2}_pool_{pool2}_npool_{npool2}_fsize_{fsize2}_psize_{psize2}_lr_1_{int(1/lr)}.pkl')
    state_dict_add = find_directory(back_num = 0, foldername = [f'{num_cycles} cycles', f'Depth Test_col_{n_ep}_{patience}', modelname, f'{rnn1}_1D CNN', 'model'], 
                                             filename = f'{rnn1}_hidden_{hid1}_nh_{nh2}_n_fil_{fil2}_pool_{pool2}_npool_{npool2}_fsize_{fsize2}_psize_{psize2}_lr_1_{int(1/lr)}_state_dict.pth')
    ta_add = find_directory(back_num = 0, foldername = [f'{num_cycles} cycles', f'Depth Test_col_{n_ep}_{patience}', modelname, f'{rnn1}_1D CNN', 'ta'],
                                             filename = f'{rnn1}_hidden_{hid1}_nh_{nh2}_n_fil_{fil2}_pool_{pool2}_npool_{npool2}_fsize_{fsize2}_psize_{psize2}_lr_1_{int(1/lr)}_ta.wb')
    ca_add = find_directory(back_num = 0, foldername = [f'{num_cycles} cycles', f'Depth Test_col_{n_ep}_{patience}', modelname, f'{rnn1}_1D CNN', 'ca'],
                                             filename = f'{rnn1}_hidden_{hid1}_nh_{nh2}_n_fil_{fil2}_pool_{pool2}_npool_{npool2}_fsize_{fsize2}_psize_{psize2}_lr_1_{int(1/lr)}_ta.wb')
    return history_add, state_dict_add, ta_add, ca_add

In [25]:
# Hyperparameters
num_vars = train_features.size()[1]

modelname = 'RNN_TA_CA_1DCNN'
rnns = ['LSTM', 'GRU', 'RNN']
hids = [3, 5, 7]
num_time = max_time
num_cycles = 100
ep_pats = [[1000, 10], [2000, 20], [3000, 500]]
lrs = [1e-4, 1e-3, 1e-2]

num_fils = [3, 5, 7]
pools = [1, 2]
npools = [1, 2]
fsize2 = 3
psize2 = 2
mids = [8, 4, 1]

nh2s = [2, 3, 4, 5]

import time

for [n_ep, patience] in ep_pats:
    print(f"epoch = {n_ep}, patience = {patience}")
    trdl = globals()[f'reg_train_dataloader_{num_cycles}']
    vdl = globals()[f'reg_val_dataloader_{num_cycles}']
    tedl = globals()[f'reg_test_dataloader_{num_cycles}']
    for rnn1, hid1 in itertools.product(rnns, hids):
        bi1 = False
        
        if rnn1.startswith('Bi'):
            bi1 = True
        for pool2, npool2 in itertools.product(pools, npools):
            print(f'Pooling: {pool2} layers, Nonpooling: {npool2} layers, split by {60/max_time} min')
            for fil2, lr in itertools.product(num_fils, lrs):
                for nh2 in nh2s:
                    print(f"rnn = {rnn1}, bi = {bi1}, hid per head = {hid1}, num_heads = {nh2}, num_fil = {fil2}, lr = {lr}")
                    
                    all_hid1 = hid1 * nh2
                    
                    start = time.time()
                    
                    # Construct CNN
                    model = globals()[modelname](num_vars, num_time, num_cycles, rnn1, bi1, all_hid1, nh2, fil2, pool2, npool2, fsize2, psize2, mids).to(device)

                    # Train
                    model_train = train(model, trdl, vdl, tedl, n_ep, lr, patience, verbose = False)
                    best_model, best_loss, history, ta, ca = model_train.TrainModel()

                    print('best_loss = '+ str(best_loss))

                    history_add, state_dict_add, ta_add, ca_add = history_state_dict_add(num_cycles, modelname, n_ep, patience,
                                                                                rnn1, hid1, nh2, fil2, pool2, npool2, fsize2, psize2, lr)

                    # saving best_epoch, loss history, ta score
                    save_data(history, history_add)
                    save_data(best_model.state_dict(), state_dict_add)
                    save_data(ta, ta_add)
                    save_data(ca, ca_add)

                    print("time: ", time.time()-start)
                    print("\n\n")

epoch = 1000, patience = 10
Pooling: 1 layers, Nonpooling: 1 layers, split by 0.5 min
rnn = LSTM, bi = False, hid per head = 3, num_heads = 2, num_fil = 3, lr = 0.0001
Early stopping at best epoch:  5
best_loss = [571.4248, 542.989, 527.2326]
time:  1.987513542175293



rnn = LSTM, bi = False, hid per head = 3, num_heads = 3, num_fil = 3, lr = 0.0001
Early stopping at best epoch:  5
best_loss = [572.0229, 543.5944, 528.09485]
time:  1.76186203956604



rnn = LSTM, bi = False, hid per head = 3, num_heads = 4, num_fil = 3, lr = 0.0001
Early stopping at best epoch:  5
best_loss = [571.2594, 542.62524, 526.94507]
time:  1.752805471420288



rnn = LSTM, bi = False, hid per head = 3, num_heads = 5, num_fil = 3, lr = 0.0001
Early stopping at best epoch:  5
best_loss = [571.16125, 542.6998, 527.0381]
time:  1.7740111351013184



rnn = LSTM, bi = False, hid per head = 3, num_heads = 2, num_fil = 3, lr = 0.001
Early stopping at best epoch:  35
best_loss = [272.49265, 183.03201, 253.81271]
time: 

Early stopping at best epoch:  29
best_loss = [289.72983, 159.50256, 177.74324]
time:  6.962863206863403



rnn = LSTM, bi = False, hid per head = 3, num_heads = 2, num_fil = 3, lr = 0.01
Early stopping at best epoch:  12
best_loss = [252.87163, 143.66603, 179.30063]
time:  3.874157190322876



rnn = LSTM, bi = False, hid per head = 3, num_heads = 3, num_fil = 3, lr = 0.01
Early stopping at best epoch:  38
best_loss = [144.97078, 104.53022, 85.564514]
time:  8.458218812942505



rnn = LSTM, bi = False, hid per head = 3, num_heads = 4, num_fil = 3, lr = 0.01
Early stopping at best epoch:  12
best_loss = [226.17033, 130.97768, 74.84743]
time:  3.8967134952545166



rnn = LSTM, bi = False, hid per head = 3, num_heads = 5, num_fil = 3, lr = 0.01
Early stopping at best epoch:  28
best_loss = [160.26248, 127.52121, 68.0763]
time:  6.802577018737793



rnn = LSTM, bi = False, hid per head = 3, num_heads = 2, num_fil = 5, lr = 0.0001
Early stopping at best epoch:  129
best_loss = [277.30267, 2

Early stopping at best epoch:  127
best_loss = [341.0334, 277.68518, 259.66306]
time:  25.036216259002686



rnn = LSTM, bi = False, hid per head = 3, num_heads = 2, num_fil = 5, lr = 0.001
Early stopping at best epoch:  5
best_loss = [572.4064, 544.07214, 528.5965]
time:  2.734700918197632



rnn = LSTM, bi = False, hid per head = 3, num_heads = 3, num_fil = 5, lr = 0.001
Early stopping at best epoch:  19
best_loss = [265.81262, 177.0588, 181.0639]
time:  5.32745361328125



rnn = LSTM, bi = False, hid per head = 3, num_heads = 4, num_fil = 5, lr = 0.001
Early stopping at best epoch:  35
best_loss = [228.64816, 156.06302, 174.27208]
time:  8.28517484664917



rnn = LSTM, bi = False, hid per head = 3, num_heads = 5, num_fil = 5, lr = 0.001
Early stopping at best epoch:  21
best_loss = [246.67798, 150.67229, 117.895966]
time:  5.593689203262329



rnn = LSTM, bi = False, hid per head = 3, num_heads = 2, num_fil = 5, lr = 0.01
Early stopping at best epoch:  5
best_loss = [571.7163, 543.2

Early stopping at best epoch:  12
best_loss = [171.62326, 136.93307, 106.43046]
time:  4.08906102180481



rnn = LSTM, bi = False, hid per head = 3, num_heads = 2, num_fil = 7, lr = 0.0001
Early stopping at best epoch:  111
best_loss = [328.59088, 251.42175, 257.75946]
time:  22.78840184211731



rnn = LSTM, bi = False, hid per head = 3, num_heads = 3, num_fil = 7, lr = 0.0001
Early stopping at best epoch:  5
best_loss = [571.3197, 542.80774, 527.2438]
time:  2.86460542678833



rnn = LSTM, bi = False, hid per head = 3, num_heads = 4, num_fil = 7, lr = 0.0001
Early stopping at best epoch:  142
best_loss = [290.46848, 235.3861, 228.0727]
time:  28.60865306854248



rnn = LSTM, bi = False, hid per head = 3, num_heads = 5, num_fil = 7, lr = 0.0001
Early stopping at best epoch:  5
best_loss = [572.56024, 544.26404, 528.7701]
time:  2.7893996238708496



rnn = LSTM, bi = False, hid per head = 3, num_heads = 2, num_fil = 7, lr = 0.001
Early stopping at best epoch:  43
best_loss = [245.06375,

Early stopping at best epoch:  21
best_loss = [233.00192, 162.8884, 130.24034]
time:  5.589321136474609



rnn = LSTM, bi = False, hid per head = 5, num_heads = 2, num_fil = 7, lr = 0.01
Early stopping at best epoch:  20
best_loss = [169.99149, 115.931885, 66.036766]
time:  5.253075838088989



rnn = LSTM, bi = False, hid per head = 5, num_heads = 3, num_fil = 7, lr = 0.01
Early stopping at best epoch:  8
best_loss = [222.25203, 110.30058, 134.18076]
time:  3.1863341331481934



rnn = LSTM, bi = False, hid per head = 5, num_heads = 4, num_fil = 7, lr = 0.01
Early stopping at best epoch:  15
best_loss = [199.70914, 144.92308, 79.08969]
time:  4.391401767730713



rnn = LSTM, bi = False, hid per head = 5, num_heads = 5, num_fil = 7, lr = 0.01
Early stopping at best epoch:  12
best_loss = [211.4379, 122.82885, 98.092186]
time:  3.9432711601257324



Pooling: 1 layers, Nonpooling: 2 layers, split by 0.5 min
rnn = LSTM, bi = False, hid per head = 5, num_heads = 2, num_fil = 3, lr = 0.0001
E

...

Early stopping at best epoch:  1868
best_loss = [199.71979, 112.59999, 115.24237]
time:  405.9239411354065



rnn = RNN, bi = False, hid per head = 5, num_heads = 3, num_fil = 3, lr = 0.0001
Early stopping at best epoch:  398
best_loss = [174.31909, 90.29391, 109.111984]
time:  156.44789600372314



rnn = RNN, bi = False, hid per head = 5, num_heads = 4, num_fil = 3, lr = 0.0001
Early stopping at best epoch:  398
best_loss = [171.76447, 94.725845, 79.57755]
time:  154.327951669693



rnn = RNN, bi = False, hid per head = 5, num_heads = 5, num_fil = 3, lr = 0.0001
Early stopping at best epoch:  529
best_loss = [165.06297, 100.02945, 73.24577]
time:  181.41195392608643



rnn = RNN, bi = False, hid per head = 5, num_heads = 2, num_fil = 3, lr = 0.001
Early stopping at best epoch:  272
best_loss = [137.56927, 85.51701, 100.07742]
time:  131.57166457176208



rnn = RNN, bi = False, hid per head = 5, num_heads = 3, num_fil = 3, lr = 0.001
Early stopping at best epoch:  403
best_loss = [97.3

Early stopping at best epoch:  130
best_loss = [114.1798, 98.873314, 106.42522]
time:  111.41854548454285



rnn = RNN, bi = False, hid per head = 5, num_heads = 3, num_fil = 3, lr = 0.01
Early stopping at best epoch:  58
best_loss = [176.49438, 107.01128, 96.299034]
time:  97.2947769165039



rnn = RNN, bi = False, hid per head = 5, num_heads = 4, num_fil = 3, lr = 0.01
Early stopping at best epoch:  1834
best_loss = [141.31172, 109.948135, 121.21813]
time:  405.8945426940918



rnn = RNN, bi = False, hid per head = 5, num_heads = 5, num_fil = 3, lr = 0.01
Early stopping at best epoch:  490
best_loss = [66.063156, 79.77534, 61.03673]
time:  173.57762837409973



rnn = RNN, bi = False, hid per head = 5, num_heads = 2, num_fil = 5, lr = 0.0001
Early stopping at best epoch:  1560
best_loss = [161.70105, 88.840324, 93.61784]
time:  366.8979744911194



rnn = RNN, bi = False, hid per head = 5, num_heads = 3, num_fil = 5, lr = 0.0001
Early stopping at best epoch:  1302
best_loss = [154.0244

Early stopping at best epoch:  255
best_loss = [130.89996, 95.339386, 75.7433]
time:  128.77833342552185



rnn = RNN, bi = False, hid per head = 7, num_heads = 3, num_fil = 5, lr = 0.001
Early stopping at best epoch:  314
best_loss = [119.35843, 83.33391, 92.744965]
time:  136.8503966331482



rnn = RNN, bi = False, hid per head = 7, num_heads = 4, num_fil = 5, lr = 0.001
Early stopping at best epoch:  234
best_loss = [121.34029, 98.68608, 95.8436]
time:  128.5748348236084



rnn = RNN, bi = False, hid per head = 7, num_heads = 5, num_fil = 5, lr = 0.001
Ends at final epoch 3000
Best epoch: 2780
best_loss = [534.2239, 477.3096, 470.2057]
time:  534.5509724617004



rnn = RNN, bi = False, hid per head = 7, num_heads = 2, num_fil = 5, lr = 0.01
Early stopping at best epoch:  1317
best_loss = [42.500412, 97.698906, 96.23848]
time:  309.0474293231964



rnn = RNN, bi = False, hid per head = 7, num_heads = 3, num_fil = 5, lr = 0.01
Early stopping at best epoch:  61
best_loss = [128.18623, 

Early stopping at best epoch:  1994
best_loss = [120.30888, 105.83283, 83.73281]
time:  435.3221263885498



rnn = RNN, bi = False, hid per head = 7, num_heads = 3, num_fil = 7, lr = 0.0001
Early stopping at best epoch:  816
best_loss = [133.8812, 85.57504, 90.539894]
time:  230.26813197135925



rnn = RNN, bi = False, hid per head = 7, num_heads = 4, num_fil = 7, lr = 0.0001
Early stopping at best epoch:  890
best_loss = [169.72014, 114.18866, 125.972305]
time:  250.28335571289062



rnn = RNN, bi = False, hid per head = 7, num_heads = 5, num_fil = 7, lr = 0.0001
Early stopping at best epoch:  1484
best_loss = [125.13587, 89.942505, 103.933815]
time:  355.73846983909607



rnn = RNN, bi = False, hid per head = 7, num_heads = 2, num_fil = 7, lr = 0.001
Early stopping at best epoch:  535
best_loss = [106.55476, 100.47334, 82.88995]
time:  181.63511729240417



rnn = RNN, bi = False, hid per head = 7, num_heads = 3, num_fil = 7, lr = 0.001
Early stopping at best epoch:  749
best_loss = [

Early stopping at best epoch:  362
best_loss = [35.44133, 71.633026, 108.780685]
time:  103.52189898490906



rnn = RNN, bi = False, hid per head = 7, num_heads = 3, num_fil = 7, lr = 0.01
Early stopping at best epoch:  690
best_loss = [48.55023, 100.516815, 79.25726]
time:  142.24085450172424



rnn = RNN, bi = False, hid per head = 7, num_heads = 4, num_fil = 7, lr = 0.01
Early stopping at best epoch:  713
best_loss = [93.912125, 113.405594, 97.30824]
time:  148.73478150367737



rnn = RNN, bi = False, hid per head = 7, num_heads = 5, num_fil = 7, lr = 0.01
Early stopping at best epoch:  299
best_loss = [142.04184, 104.97621, 84.90186]
time:  97.32103204727173



Pooling: 2 layers, Nonpooling: 2 layers, split by 0.5 min
rnn = RNN, bi = False, hid per head = 7, num_heads = 2, num_fil = 3, lr = 0.0001
Early stopping at best epoch:  1476
best_loss = [146.7762, 86.573425, 100.30865]
time:  249.10121941566467



rnn = RNN, bi = False, hid per head = 7, num_heads = 3, num_fil = 3, lr = 0.0

In [ ]:
modelname = 'RNN_TA_CA_1DCNN'
rnns = ['LSTM', 'GRU', 'RNN']
hids = [3, 5, 7]
num_time = max_time
num_cycles = 100
ep_pats = [[1000, 10], [2000, 20], [3000, 500]]
lrs = [1e-4, 1e-3, 1e-2]

num_fils = [3, 5, 7]
pools = [1, 2]
npools = [1, 2]
fsize2 = 3
psize2 = 2
mids = [8, 4, 1]

nh2s = [2, 3, 4, 5]

for [n_ep, patience] in ep_pats:
    for nh2 in nh2s:
        for rnn1 in rnns:
            df = pd.DataFrame(columns = ['train rmse', 'val rmse', 'test rmse'])
            bi1 = False
            if rnn1.startswith('Bi'):
                bi1 = True
            for hid1 in hids:
                for pool2, npool2 in itertools.product(pools, npools):
                    for fil2, lr in itertools.product(num_fils, lrs):
                        history_add, state_dict_add, ta_add, ca_add = history_state_dict_add(num_cycles, modelname, n_ep, patience,
                                                                                    rnn1, hid1, nh2, fil2, pool2, npool2, fsize2, psize2, lr)

                        history = load_data(history_add)

                        df.loc[f'{rnn1}_{hid1}_{fil2}_{pool2}_{npool2}_{fsize2}_{lr}'] = [history.iloc[-1, 0], history.iloc[-1, 1], history.iloc[-1, 2]]

        print(n_ep, patience, rnn1)
        df
        df_add = find_directory(back_num = 0, foldername = [f'{num_cycles} cycles', f'Depth Test_col_{n_ep}_{patience}', modelname], 
                        filename = f'{modelname}_{rnn1}_{nh2}.csv')
        df.to_csv(df_add)

In [ ]:
# Hyperparameters
num_vars = train_features.size()[1]

modelname = 'RNN_TA_CA_1DCNN'
rnns = ['BiLSTM', 'BiGRU', 'BiRNN']
hids = [3, 5, 7]
num_time = max_time
num_cycles = 100
ep_pats = [[1000, 10], [2000, 20], [3000, 500]]
lrs = [1e-4, 1e-3, 1e-2]

num_fils = [3, 5, 7]
pools = [1, 2]
npools = [1, 2]
fsize2 = 3
psize2 = 2
mids = [8, 4, 1]

nh2s = [2, 3, 4, 5]

import time

for [n_ep, patience] in ep_pats:
    print(f"epoch = {n_ep}, patience = {patience}")
    trdl = globals()[f'reg_train_dataloader_{num_cycles}']
    vdl = globals()[f'reg_val_dataloader_{num_cycles}']
    tedl = globals()[f'reg_test_dataloader_{num_cycles}']
    for rnn1, hid1 in itertools.product(rnns, hids):
        bi1 = False
        
        if rnn1.startswith('Bi'):
            bi1 = True
        for pool2, npool2 in itertools.product(pools, npools):
            print(f'Pooling: {pool2} layers, Nonpooling: {npool2} layers, split by {60/max_time} min')
            for fil2, lr in itertools.product(num_fils, lrs):
                for nh2 in nh2s:
                    start = time.time()
                    print(f"rnn = {rnn1}, bi = {bi1}, hid = {hid1}, num_heads = {nh2}, num_fil = {fil2}, lr = {lr}")

                    # Construct CNN
                    model = globals()[modelname](num_vars, num_time, num_cycles, rnn1, bi1, hid1, nh2, fil2, pool2, npool2, fsize2, psize2, mids).to(device)

                    # Train
                    model_train = train(model, trdl, vdl, tedl, n_ep, lr, patience, verbose = False)
                    best_model, best_loss, history, ta, ca = model_train.TrainModel()

                    print('best_loss = '+ str(best_loss))

                    history_add, state_dict_add, ta_add, ca_add = history_state_dict_add(num_cycles, modelname, n_ep, patience,
                                                                                rnn1, hid1, nh2, fil2, pool2, npool2, fsize2, psize2, lr)

                    # saving best_epoch, loss history, ta score
                    save_data(history, history_add)
                    save_data(best_model.state_dict(), state_dict_add)
                    save_data(ta, ta_add)
                    save_data(ca, ca_add)

                    print("time: ", time.time()-start)
                    print("\n\n")